In [3]:
import numpy as np
import pandas as pd

# Import Data Set from given Excel sheet
bitcoin_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 0)
ethereum_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 1)
doge_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 2)
gold_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 3)
SP500_data = pd.read_excel('Asset Pricing Data.xlsx', sheet_name = 4)

In [1]:
#bitcoin_data.head()
#ethereum_data.head()
#doge_data.head()
#gold_data.info()
#SP500_data.head()

### Cleaning the Data Set before Exploration

In [4]:
# To remove one row where all values are '-'

gold_data = gold_data.drop(gold_data[gold_data.Date == "29/1/2018"].index)

# To remove 4 rows where corresponding cryptocurrencies do not have close value
gold_data = gold_data.drop(gold_data[gold_data.Date == "2020/10/13"].index)
gold_data = gold_data.drop(gold_data[gold_data.Date == "2020/10/12"].index)
gold_data = gold_data.drop(gold_data[gold_data.Date == "2020/10/9"].index)
gold_data = gold_data.drop(gold_data[gold_data.Date == "2020/4/17"].index)

SP500_data = SP500_data.drop(SP500_data[SP500_data.Date == "2020/10/13"].index)
SP500_data = SP500_data.drop(SP500_data[SP500_data.Date == "2020/10/12"].index)
SP500_data = SP500_data.drop(SP500_data[SP500_data.Date == "2020/10/9"].index)
SP500_data = SP500_data.drop(SP500_data[SP500_data.Date == "2020/4/17"].index)

### Extracting the Closing price values for each asset class

In [5]:
bitcoin_close = pd.DataFrame(bitcoin_data[["Date", "Close"]])
ethereum_close = pd.DataFrame(ethereum_data[["Date", "Close"]])
doge_close = pd.DataFrame(doge_data[["Date", "Close"]])
gold_data["Close"] = gold_data["Close"].astype('float')
gold_close = pd.DataFrame(gold_data[["Date", "Close"]])

SP500_close = pd.DataFrame(SP500_data[["Date", "Close"]])

### Combining data for the 3 crytocurrencies

In [6]:
# Merge BTC and ETH
crypto_data = bitcoin_close.merge(ethereum_close, left_on='Date', right_on='Date')
crypto_data = crypto_data.rename(columns={"Close_x" : "BTC_Close", "Close_y" : "ETH_Close"})


# Merge Doge with existing 2 asset classes
crypto_data = crypto_data.merge(doge_close, left_on='Date', right_on='Date')
crypto_data = crypto_data.rename(columns={"Close" : "Doge_Close"})

In [7]:
crypto_data = crypto_data.set_index('Date')

In [9]:
#crypto_data.corr()

## Since the weekly trading window for Gold and S&P500 differ, we will compare the group of cryptocurrencies against each asset separately

## Set 1 : Crypto + Gold

In [11]:
# Merge Gold with existing 3 crypto assets
crypto_gold = crypto_data.merge(gold_close, how='left', left_on='Date', right_on='Date')
crypto_gold = crypto_gold.rename(columns={"Close" : "Gold_Close"})


crypto_gold = crypto_gold.set_index('Date')
crypto_gold.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2021-12-15 to 2016-12-15
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BTC_Close   1823 non-null   float64
 1   ETH_Close   1823 non-null   float64
 2   Doge_Close  1823 non-null   float64
 3   Gold_Close  1257 non-null   float64
dtypes: float64(4)
memory usage: 71.4 KB


In [12]:
# Perform correlation on Logarithmic values to reduce disparity of prices between assets

log_crypto_gold = np.log(crypto_gold/crypto_gold.shift())

log_crypto_gold.corr()

,BTC_Close,ETH_Close,Doge_Close,Gold_Close
BTC_Close,1.000000,0.689398,0.467352,0.076692
ETH_Close,0.689398,1.000000,0.444276,0.072958
Doge_Close,0.467352,0.444276,1.000000,0.027722
Gold_Close,0.076692,0.072958,0.027722,1.000000


## Set 2 : Crypto + S&P500

In [13]:
# Merge S&P500 with existing 3 crypto assets
crypto_sp500 = crypto_data.merge(SP500_close, how='left', left_on='Date', right_on='Date')
crypto_sp500 = crypto_sp500.rename(columns={"Close" : "S&P500_Close"})

crypto_sp500 = crypto_sp500.set_index('Date')
crypto_sp500.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2021-12-15 to 2016-12-15
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BTC_Close     1823 non-null   float64
 1   ETH_Close     1823 non-null   float64
 2   Doge_Close    1823 non-null   float64
 3   S&P500_Close  1255 non-null   float64
dtypes: float64(4)
memory usage: 71.4 KB


In [14]:
# Perform correlation on Logarithmic values to reduce disparity of prices between assets

log_crypto_sp500 = np.log(crypto_sp500/crypto_sp500.shift())

log_crypto_sp500.corr()

,BTC_Close,ETH_Close,Doge_Close,S&P500_Close
BTC_Close,1.000000,0.689398,0.467352,0.220237
ETH_Close,0.689398,1.000000,0.444276,0.231029
Doge_Close,0.467352,0.444276,1.000000,0.110418
S&P500_Close,0.220237,0.231029,0.110418,1.000000
